In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams['figure.figsize']=(20,5)
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import pyarrow.parquet as pq

In [2]:
def sliderPlot(df1,i):
    fig = px.line(df1, y=df1[i] )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=2, label="2y", step="year", stepmode="backward"),
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ))
    fig.show()

In [3]:
table = pq.read_table(r'C:/Users/Harsh/Downloads/DLMS Data/data.parquet')

# Convert it to a Pandas DataFrame (optional)
dframe = table.to_pandas()

In [4]:
dframe.head(1)

,sensor,Clock,R_Voltage,Y_Voltage,B_Voltage,R_Current,Y_Current,B_Current,Kwh
0,5f718c439c7a78.65267835,2022-11-18 00:30:00,6499.0,6514.0,6473.0,27.74,27.42,25.57,261300.0


In [5]:
id=dframe['sensor'].unique()
id

array(['5f718c439c7a78.65267835', '62a9920f75c931.62399458'], dtype=object)

In [6]:
lst=[]
for location in id:
    df=dframe[dframe['sensor'] == location ]
    # df['Clock']=pd.to_datetime(df['Clock'])
    df.set_index(['Clock'],inplace=True,drop= True)
    df = df[df.index >= '2022-11-18 00:00:00']
    filtered_df = df[((df['R_Voltage'] == 0) | (df['Y_Voltage'] == 0) | (df['B_Voltage'] == 0)) & 
                     ((df['R_Current'] == 0) | (df['Y_Current'] == 0) | (df['B_Current'] == 0))]
    filtered_df['Kwh'] = 0
    df.loc[filtered_df.index, :] = filtered_df
    df=df[['Kwh']].resample(rule='1H').sum()
    df['id']=location
    df['kw_roll']=df['Kwh'].rolling(window=24).mean()
    df.dropna(inplace=True)

    lst.append(df)

In [7]:
lst[0]

,Kwh,id,kw_roll
Clock,,,
2022-11-18 23:00:00,665200.0,5f718c439c7a78.65267835,717695.833333
2022-11-19 00:00:00,592300.0,5f718c439c7a78.65267835,731487.500000
2022-11-19 01:00:00,526300.0,5f718c439c7a78.65267835,732204.166667
2022-11-19 02:00:00,505700.0,5f718c439c7a78.65267835,733212.500000
2022-11-19 03:00:00,487400.0,5f718c439c7a78.65267835,733845.833333
...,...,...,...
2023-10-31 22:00:00,1066900.0,5f718c439c7a78.65267835,985366.666667
2023-10-31 23:00:00,975500.0,5f718c439c7a78.65267835,983391.666667
2023-11-01 00:00:00,887000.0,5f718c439c7a78.65267835,981958.333333


In [8]:
lst[1]

,Kwh,id,kw_roll
Clock,,,
2022-12-03 23:00:00,208800.0,62a9920f75c931.62399458,248087.500000
2022-12-04 00:00:00,169900.0,62a9920f75c931.62399458,251579.166667
2022-12-04 01:00:00,158000.0,62a9920f75c931.62399458,251862.500000
2022-12-04 02:00:00,142800.0,62a9920f75c931.62399458,252229.166667
2022-12-04 03:00:00,135000.0,62a9920f75c931.62399458,251845.833333
...,...,...,...
2023-11-01 08:00:00,320800.0,62a9920f75c931.62399458,256695.833333
2023-11-01 09:00:00,337300.0,62a9920f75c931.62399458,256479.166667
2023-11-01 10:00:00,301500.0,62a9920f75c931.62399458,255737.500000
